**Importamos las librerias**

In [47]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib

**Cargamos los datos**

In [58]:
df = pd.read_csv('../_src/Datasets/movies_model.csv')
df.shape

(14879, 5)

Nos quedamos con las primeras 7000 peliculas

In [49]:
df = df.head(7000)

In [50]:
df["genres"] = df["genres"].fillna('').astype(str)
df["production_companies"] = df["production_companies"].fillna('').astype(str)
df["title"] = df["title"].fillna('').astype(str)
df["crew"] = df["crew"].fillna('').astype(str)

# Concatenamos las columnas
combined_text = df["genres"] + " " + df["production_companies"] # + " " + df["crew"] + " " + df["title"]
# Tras algunas pruebas posteriores he decidido que crew y title no son buenas columnas para las recomendaciones
combined_text = combined_text.str.replace(',','')

In [51]:
# Instanciamos el modelo
vectorizer = CountVectorizer()
# Entrenamos el modelo
classification_matrix = vectorizer.fit_transform(combined_text)
# Calculamos la similitud del coseno
cosine_sim = cosine_similarity(classification_matrix)

Guardamos la matrix de similitud en un archivo comprimido de joblib

In [52]:
joblib.dump(cosine_sim, '../_src/Datasets/cosine_sim.joblib', compress=True)

['./Dataset/cosine_sim.joblib']

Creamos la funcion de recomendacion y la testeamos 

In [53]:
def recomendacion(titulo: str, n: int = 5):
    # Obtenemos el índice de la película en el DataFrame
    indice = df[df['title'] == titulo].index[0]
    # Obtenemos los scores de similitud para la película seleccionada
    sim_scores = list(enumerate(cosine_sim[indice]))
    # Ordenamos los scores de mayor a menor
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Creamos la lista donde pondremos las n peliculas mas similares
    recomendaciones = []
    i = 0
    while len(recomendaciones)!=n:
        if df.iloc[sim_scores[i][0]].title == titulo:
                i+=1
        recomendaciones.append((df.iloc[sim_scores[i][0]].title))
        i+=1
    return {"Pelicula":titulo,"Recomendaciones":recomendaciones}

In [54]:
recomendacion("Cars")

{'Pelicula': 'Cars',
 'Recomendaciones': ["A Bug's Life",
  'Monsters, Inc.',
  'The Incredibles',
  'Ratatouille',
  'Meet the Robinsons']}